## Package Imports

In [14]:
#multiprocess inference: currently, there is one analyze process running throughohut the duration of the program
    #main adds the microphone_data to a queue which analyze gets from the queue and analyzes


import pyaudio
import librosa
import logging
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from multiprocessing import Process, active_children
from tensorflow.keras import Input, layers, optimizers, backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import IPython.display as ipd
import matplotlib.pyplot as plt
#from gsmmodem.modem import GsmModem
stream_handler = logging.StreamHandler(sys.stdout)
import time

In [15]:
import multiprocessing, logging
logger = multiprocessing.log_to_stderr()
logger.setLevel(multiprocessing.SUBDEBUG)

## Variable Initializations

In [16]:
#stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger('debugger')
logger.setLevel(logging.DEBUG)
ch = logging.FileHandler('spam.log')
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
emit_string = "send message here"
logger.debug(emit_string)

In [17]:
audio_format = pyaudio.paFloat32
audio_rate = 44100
audio_channels = 1
audio_device_index = 0
audio_frames_per_buffer = 4410
audio_sample_duration = 2
input_shape = (audio_rate, 1)
modem_port = '/dev/ttyUSB0'
modem_baudrate = 115200
modem_sim_pin = None # SIM card PIN (if any)
phone_numbers_to_message = ["8163449956", "9176202840", "7857642331"]

## Establishing a Connection to the SMS Modem

In [ ]:
#print("Initializing connection to modem...")
#modem = GsmModem(modem_port, modem_baudrate)
#modem.smsTextMode = False
#modem.connect(modem_sim_pin)

## ROC (AUC) metric - Uses the import "from tensorflow.keras import backend as K"

In [ ]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

## Original Model Parameters

In [ ]:
drop_out_rate = 0.1
learning_rate = 0.001
number_of_epochs = 100
number_of_classes = 2
batch_size = 32
optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
input_tensor = Input(shape=input_shape)
metrics = [auc, "accuracy"]

## Loading Original Model

In [ ]:
x = layers.Conv1D(16, 9, activation="relu", padding="same")(input_tensor)
x = layers.Conv1D(16, 9, activation="relu", padding="same")(x)
x = layers.MaxPool1D(16)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPool1D(4)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPool1D(4)(x)
x = layers.Dropout(rate=drop_out_rate)(x)

x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(1028, activation="relu")(x)
output_tensor = layers.Dense(number_of_classes, activation="softmax")(x)

model = tf.keras.Model(input_tensor, output_tensor)
model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)

## Loading Original Model Weights

In [ ]:
model.load_weights("./models/gunshot_sound_model.h5")

## Defining Thread Functions

In [18]:
def analyze_microphone_data(audio_rate, phone_numbers_to_message):
    #removed modem, model, microphone_data from parameters

    def auc(y_true, y_pred):
        auc = tf.metrics.auc(y_true, y_pred)[1]
        K.get_session().run(tf.local_variables_initializer())
        return auc

    drop_out_rate = 0.1
    learning_rate = 0.001
    number_of_epochs = 100
    number_of_classes = 2
    batch_size = 32
    optimizer = optimizers.Adam(learning_rate, learning_rate / 100)
    input_tensor = Input(shape=input_shape)
    metrics = [auc, "accuracy"]
    
    x = layers.Conv1D(16, 9, activation="relu", padding="same")(input_tensor)
    x = layers.Conv1D(16, 9, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(16)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool1D(4)(x)
    x = layers.Dropout(rate=drop_out_rate)(x)

    x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(256, 3, activation="relu", padding="same")(x)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Dropout(rate=(drop_out_rate * 2))(x) # Increasing drop-out rate here to prevent overfitting

    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(1028, activation="relu")(x)
    output_tensor = layers.Dense(number_of_classes, activation="softmax")(x)

    model = tf.keras.Model(input_tensor, output_tensor)
    model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=metrics)
    
    model.load_weights("./models/gunshot_sound_model.h5")

    while True:
        microphone_data = q.get()
        
        # Performs post-processing on live audio samples
        reformed_microphone_data = librosa.resample(y=microphone_data, orig_sr=audio_rate, target_sr=22050)
        reformed_microphone_data = librosa.util.normalize(reformed_microphone_data)
        reformed_microphone_data = reformed_microphone_data[:audio_rate]
        reformed_microphone_data = reformed_microphone_data.reshape(-1, audio_rate, 1)

        # Passes a given audio sample into the model for prediction
        probabilities = model.predict(reformed_microphone_data)
        emit_string = "Probabilities derived by the model: " + str(probabilities)
        logger.debug(emit_string)

        if (probabilities[0][1] >= 0.9):
            sms_alert_process = Process(target = send_sms_alert, args = (probabilities))
            sms_alert_process.start()


def send_sms_alert(probabilities):
        # If the model detects a gunshot, an SMS alert will be sent to local authorities
        logger.debug("pretend like I just sent a text message")
        '''
        try:
            modem.waitForNetworkCoverage(timeout=86400)
            message = " (Testing) ALERT: A Gunshot Has Been Detected (Testing)"
            for number in phone_numbers_to_message:
                modem.sendSms(number, message)
            stream_handler.emit(" *** Sent out an SMS alert to all designated recipients *** ")
        except:
            stream_handler.emit("ERROR: Unable to successfully send an SMS alert to the designated recipients.")
            pass
        finally:
            stream_handler.emit(" * Finished evaluating an audio sample with the model * ")
        '''

## Capturing Microphone Audio

In [19]:
#open the stream
pa = pyaudio.PyAudio()
    
stream = pa.open(format = audio_format,
                 rate = audio_rate,
                 channels = audio_channels,
                 input_device_index = audio_device_index,
                 frames_per_buffer = audio_frames_per_buffer,
                 input = True)

In [ ]:
#testing audio input
np_array_data = []

# Loops through the stream and appends audio chunks to the frame array
for i in range(0, int(audio_rate / audio_frames_per_buffer * audio_sample_duration)):
    data = stream.read(audio_frames_per_buffer, exception_on_overflow = False)
    np_array_data.append(np.frombuffer(data, dtype=np.float32))
microphone_data = np.concatenate(np_array_data)

print(len(microphone_data))

ipd.Audio(microphone_data, rate=audio_rate)

In [20]:
if __name__ == '__main__':
    
    q = multiprocessing.Queue()
    analysis_process = Process(target = analyze_microphone_data, args = (audio_rate, phone_numbers_to_message))
    analysis_process.start()

    logger.debug("--- Recording Audio ---")
    while(True):
        np_array_data = []

        # Loops through the stream and appends audio chunks to the frame array
        for i in range(0, int(audio_rate / audio_frames_per_buffer * audio_sample_duration)):
            data = stream.read(audio_frames_per_buffer, exception_on_overflow = False)
            np_array_data.append(np.frombuffer(data, dtype=np.float32))
        microphone_data = np.concatenate(np_array_data)
        emit_string = "Cumulative length of a given two-second audio sample: " + str(len(microphone_data))
        logger.debug(emit_string)
        emit_string = "The maximum frequency value for a given two-second audio sample: " + str(max(microphone_data))
        logger.debug(emit_string)
        


        # If a sample meets a certain threshold, a new concurrent analysis process is created
        if max(microphone_data) >= 0.001:
            q.put(microphone_data)
            

        # Closes all finished processes    
        kids = active_children()
        logger.debug("these are my active children: ")
        logger.debug(kids)

[DEBUG/MainProcess] created semlock with handle 90
[DEBUG/MainProcess] created semlock with handle 90
[DEBUG/MainProcess] created semlock with handle 89
[DEBUG/MainProcess] created semlock with handle 89
[DEBUG/MainProcess] created semlock with handle 91
[DEBUG/MainProcess] created semlock with handle 91
[DEBUG/MainProcess] Queue._after_fork()
[DEBUG/MainProcess] Queue._after_fork()
[Level 5/MainProcess] finalizer calling <function Queue._finalize_close at 0x114ff1ea0> with args [deque([]), <Condition(<unlocked _thread.lock object at 0x134ebdc88>, 1)>] and kwargs {}
[Level 5/MainProcess] finalizer calling <function Queue._finalize_close at 0x114ff1ea0> with args [deque([]), <Condition(<unlocked _thread.lock object at 0x134ebdc88>, 1)>] and kwargs {}
[DEBUG/MainProcess] telling queue thread to quit
[DEBUG/MainProcess] telling queue thread to quit
[DEBUG/MainProcess] feeder thread got sentinel -- exiting
[DEBUG/MainProcess] feeder thread got sentinel -- exiting
[Level 5/MainProcess] fina

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


[DEBUG/MainProcess] Queue._start_thread()
[DEBUG/MainProcess] Queue._start_thread()
[DEBUG/MainProcess] doing self._thread.start()
[DEBUG/MainProcess] doing self._thread.start()
[DEBUG/MainProcess] starting thread to feed data to pipe
[DEBUG/MainProcess] starting thread to feed data to pipe
[DEBUG/MainProcess] ... done self._thread.start()
[DEBUG/MainProcess] ... done self._thread.start()
[Level 5/MainProcess] finalizer calling <built-in function close> with args (73,) and kwargs {}
[Level 5/MainProcess] finalizer calling <built-in function close> with args (73,) and kwargs {}
[DEBUG/Process-19:1] Queue._after_fork()
[DEBUG/Process-19:1] Queue._after_fork()
[INFO/Process-19:1] child process calling self.run()
[INFO/Process-19:1] child process calling self.run()
[INFO/Process-19:1] process shutting down
[INFO/Process-19:1] process shutting down
[DEBUG/Process-19:1] running all "atexit" finalizers with priority >= 0
[DEBUG/Process-19:1] running all "atexit" finalizers with priority >= 0


KeyboardInterrupt: 

[Level 5/Process-19] finalizer no longer registered
[INFO/Process-19] process exiting with exitcode 1
[INFO/Process-19] process exiting with exitcode 1
